In [1]:
import os
from pathlib import Path

from arcgis.features import GeoAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
project_parent = Path('./').absolute().parent
data_dir = project_parent/'data'
data_raw = data_dir/'raw'
gdb_raw = data_raw/'raw.gdb'

# load environment variables from .env
load_dotenv(find_dotenv())

# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=os.getenv('ESRI_GIS_PASSWORD')
)

gis

GIS @ https://geoai.maps.arcgis.com version:8.4

In [3]:
sg_poi_id = '3425348e4bee4059af2b353e52df43c2'

In [4]:
sg_poi_lyr = gis.content.get(sg_poi_id).layers[0]
sg_poi_lyr

<FeatureLayer url:"https://services7.arcgis.com/1EHlBtDZfCcbIqb1/arcgis/rest/services/aaa19f/FeatureServer/0">

In [5]:
sg_cat = 'Museums, Historical Sites, and Similar Institutions'
sg_cty = 'Seattle'
sg_st = 'WA'

sql = f"""topCat = '{sg_cat}' AND city = '{sg_cty}' AND state = '{sg_st}'"""

In [15]:
sg_poi_df = sg_poi_lyr.query(sql).sdf

sg_poi_df.head()

,FID,sgpid,psgpid,sgbid,poiName,brands,topCat,latitude,longitude,address,city,state,zip_code,placekey,iso_country_code,SHAPE
0,307,sg:170f22677e074ceaa66c2b3f97a78510,null,null,Diagonal Ave South Public Shoreline Access,null,"Museums, Historical Sites, and Similar Institu...",47.560730,-122.343707,4691 4699 Diagonal Ave S,Seattle,WA,98134,zzy-222@5x4-4vw-y7q,US,"{""x"": -13619239.16500146, ""y"": 6034084.6193307..."
1,20300,sg:2ebe2cebb284471cb13ee9a9f55aa69f,null,null,Ella Bailey Park,null,"Museums, Historical Sites, and Similar Institu...",47.640686,-122.391383,2601 West Smith St,Seattle,WA,98199,zzz-222@5x4-4fy-y9z,US,"{""x"": -13624546.433044516, ""y"": 6047284.608349..."
2,43145,sg:b5c11e760ee343939a74857c92ee1521,null,null,Stanley Sayres Memorial Park,null,"Museums, Historical Sites, and Similar Institu...",47.570330,-122.277515,3800 Lake Washington Blvd S,Seattle,WA,98118,zzz-222@5x4-49q-jn5,US,"{""x"": -13611870.70526687, ""y"": 6035668.4246060..."
3,48959,sg:8496210929b844d197a933b3ef49db8f,null,null,Hing Hay Park,null,"Museums, Historical Sites, and Similar Institu...",47.598607,-122.325435,414 6th Ave S,Seattle,WA,98104,zzz-222@5x4-4vt-5zz,US,"{""x"": -13617205.13526568, ""y"": 6040335.2437240..."
4,64647,sg:a2f401845446427bb425c46e4cd7b9dd,sg:e487863dfcd04e5d91d53d7b157fed5a,null,Gilman Playground,null,"Museums, Historical Sites, and Similar Institu...",47.667418,-122.369522,923 NW 54th St,Seattle,WA,98107,zzy-222@5x4-4g3-9j9,US,"{""x"": -13622112.877656283, ""y"": 6051702.317211..."


In [27]:
poi_park = sg_poi_df[sg_poi_df.poiName.str.contains('Green Lake Park')].iloc[0]

poi_park

FID                                                            860879
sgpid                             sg:019e60dd8d984b69994c37267bc103ad
psgpid                                                           null
sgbid                                                            null
poiName                                               Green Lake Park
brands                                                           null
topCat              Museums, Historical Sites, and Similar Institu...
latitude                                                      47.6778
longitude                                                    -122.339
address                                               Green Lake Park
city                                                          Seattle
state                                                              WA
zip_code                                                        98103
placekey                                          zzz-222@5x4-48g-q75
iso_country_code    

In [29]:
sg_poi_df[sg_poi_df.sgbid == poi_park.sgpid]

,FID,sgpid,psgpid,sgbid,poiName,brands,topCat,latitude,longitude,address,city,state,zip_code,placekey,iso_country_code,SHAPE


In [7]:
sg_poi_df.spatial.to_featureclass(gdb_raw/'sg_poi_seattle_rec')

'D:\\projects\\uw-parks\\data\\raw\\raw.gdb\\sg_poi_seattle_rec'